In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.applications.inception_v3 import preprocess_input

2021-08-13 15:30:51.290487: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-13 15:30:51.290552: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# WE ARE TRAINING ON FIVE CLASSES/ ACTORS 
1. We Are training the model to recognise the five actors who played superman in the last 2 decades
2. The folder structure is as follows
            supermen/
                /cavill
                /welling
                /hoeclin
                /reeves
                /routh

3. Each of these subdirectories acts as a label for the dataset
4. I took a single picture and augmented it to get a larger dataset
5. The augmentedModel.py file is given the name of the image, which has to be one of the subdirectory name, ending in a '.jpg' extension. This file resides in the smallClassify or the root directory
6. Then this script 'augmentedModel.py' is executed the folders are populated with augmented dataset
7. Do this for each of the actors. Change the filename, execute 'augmentedModel.py' script. Repeat.
8. Then See the JupyterNotebook for Comments

# Setting up the training generators

The training and validation generators are created. THese supply images to the model one by one.
We split it into 80% training 20% validation set, see validtion_split. The classes are split into categorical mode to show that they are separate classes. Then the image size is set, for this InceptionV3 model, the INPUT Image Size has to be 299 x 299 x 3, i.e. three channels for color, also rescale the images to have pixel values from 0 to 255, this is important. Give it the training_dataset directory that we made using the augmented_Model.py script.




In [94]:

train_data_dir = 'supermen'
batch_size = 16
img_height = 299
img_width = 299
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 184 images belonging to 5 classes.
Found 45 images belonging to 5 classes.


In [72]:
# CREATE MODEL FROM INCEPTION_V3
# base_model loads the InceptionV3 model
# predictions add a new Dense layer with 5 neurons for our five classes
#then the model is create using base_model and predictions as its output

base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [73]:
# LOAD SAVED MODEL
# to load a saved model uncomment this and run it 
#model = keras.models.load_model('mymodel')
# model.summary()

In [ ]:
# Layers are set to not trainable
# we set original InceptionV3 model to freeze, so that out Dense layer has a chance to get initialised

for layer in model.layers:
    layer.trainable = False


model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# start fitting the model to get Dense layer initialsed
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // batch_size,
    epochs = 10)

In [100]:
# Print layer names and then set the last layer few layers to unfreeze so we can train them to classify

for i, layer in enumerate(model.layers):
    print(i, layer.name)

for layer in model.layers[:249]:
    layer.trainable = False

for layer in model.layers[249:]:
    layer.trainable = True
    print(layer.name)
 


0 input_1
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_14
5

In [96]:
# Compile our Model to use Stochastic Gradient Descent Algorithm with a very low Learning Rate so it can stop Underfitting

from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy')

model.fit_generator(
    
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // batch_size,
    epochs = 30)

Epoch 1/30
11/11 [==============================] - 20s 2s/step - loss: 1.2015 - val_loss: 1.1771
Epoch 2/30
11/11 [==============================] - 16s 1s/step - loss: 0.9015 - val_loss: 0.9425
Epoch 3/30
11/11 [==============================] - 16s 1s/step - loss: 0.7145 - val_loss: 0.6704
Epoch 4/30
11/11 [==============================] - 16s 1s/step - loss: 0.5891 - val_loss: 0.5146
Epoch 5/30
11/11 [==============================] - 16s 2s/step - loss: 0.5032 - val_loss: 0.4410
Epoch 6/30
11/11 [==============================] - 17s 2s/step - loss: 0.4243 - val_loss: 0.4082
Epoch 7/30
11/11 [==============================] - 17s 2s/step - loss: 0.3590 - val_loss: 0.3564
Epoch 8/30
11/11 [==============================] - 17s 2s/step - loss: 0.3159 - val_loss: 0.2524
Epoch 9/30
11/11 [==============================] - 17s 2s/step - loss: 0.2989 - val_loss: 0.2437
Epoch 10/30
11/11 [==============================] - 17s 2s/step - loss: 0.2873 - val_loss: 0.2483
Epoch 11/30
11/11 [

In [79]:
#Save the model as retrained_model
model.save('retrained_model')

2021-08-13 17:07:34.375263: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: retrained_model/assets


# IGNORE THIS CELL

In [101]:
# test_data_dir = 'test'


# test_datagen = ImageDataGenerator(
#     rescale=1./255,
#     ) # set validation split




# test_generator = test_datagen.flow_from_directory(
#     test_data_dir,
#     target_size=(img_height, img_width),
#     batch_size=1,
#     class_mode='categorical',
#     subset='validation') # set as test data
# unseen_img = '.jpeg'
# img = image.load_img(unseen_img, target_size=(img_height, img_width))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)
# y_prob = model.predict(x)
# list(train_generator.class_indices)[np.argmax(y_prob)]

# Test the model on test set

input images into the test folder as is done in supermen directory
and run this block
It outputs what the Predicted Result is Versus the Actual Name

N.B I need to code it to give accrucy values, for now the accuracy is around 30%

In [92]:

import glob

for filepath in glob.iglob('./test/*/*'):
    
    img = image.load_img(filepath, target_size=(img_height, img_width))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    y_prob = model.predict(x)
    print(list(train_generator.class_indices)[np.argmax(y_prob)], filepath[14:])
    # print(y_prob, np.argmax(y_prob)+1, filepath[14:])


[[0.15501486 0.21800774 0.10289738 0.16331369 0.3607663 ]] 5 /tyler3.jpeg
[[0.12466992 0.34644377 0.16873373 0.17250645 0.18764617]] 2 /tyler.jpeg
[[0.01094298 0.9544403  0.01444672 0.00981366 0.01035633]] 2 /hoeclin.jpg
[[0.16317339 0.18183959 0.21899135 0.22014625 0.21584934]] 4 /tyler2.jpeg
[[0.25186446 0.30698174 0.09546199 0.2207594  0.1249324 ]] 2 /welling224.jpeg
[[0.12573938 0.26510295 0.26250795 0.25101313 0.09563664]] 2 /welling3444.jpeg
[[0.18175295 0.3003572  0.09247416 0.2601252  0.16529052]] 2 /welling (1).jpeg
[[0.31992063 0.23612274 0.16281693 0.15921114 0.12192853]] 1 /welling.jpeg
[[0.09524105 0.2261723  0.2834292  0.19061413 0.2045433 ]] 3 /welling211113.jpeg
[[0.27116245 0.14984548 0.29829383 0.16736977 0.11332853]] 3 outh.jpeg
[[0.34925246 0.239457   0.13798626 0.15341815 0.11988615]] 1 outh2.jpeg
[[0.28725067 0.28023827 0.17587826 0.16196305 0.09466974]] 1 cavill23556.jpeg
[[0.1849157  0.18616256 0.10924022 0.13056192 0.38911954]] 5 cavill2.jpg
[[0.15344335 0.1067

In [88]:
train_generator.class_indices

{'cavill': 0, 'hoeclin': 1, 'reeves': 2, 'routh': 3, 'welling': 4}

In [30]:
model.save('mymodel')

2021-08-13 06:41:44.552922: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: mymodel/assets
